# Using phase and gesture to predict the gesture

In [49]:
import pandas as pd
import numpy as np
import warnings

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv("data/train.csv")
train_demo = pd.read_csv("data/train_demographics.csv")

In [26]:
def extract_features(df: pd.DataFrame):
    features = []

    for seq_id, group in df.groupby('sequence_id'):
        feats = {'sequence_id': seq_id}

        for phase, phase_group in group.groupby('phase'):
        

            # Accelerometer features
            for axis in ['acc_x', 'acc_y', 'acc_z']:
                feats[f'{axis}_{phase}_mean'] = phase_group[axis].mean()
                feats[f'{axis}_{phase}_std'] = phase_group[axis].std()
                feats[f'{axis}_{phase}_max'] = phase_group[axis].max()
                feats[f'{axis}_{phase}_min'] = phase_group[axis].min()
    
            # Thermopile sensors
            for i in range(1, 6):
                col = f'thm_{i}'
                feats[f'{phase}_{col}_mean'] = phase_group[col].mean()
                feats[f'{phase}_{col}_std'] = phase_group[col].std()

            # ToF: Count close pixels & missing data
            for s in range(1, 6):
                tof_cols = [f'tof_{s}_v{v}' for v in range(64)]
                vals = phase_group[tof_cols].values
                close_mask = (vals >= 0) & (vals < 50)
                feats[f'{phase}_tof_{s}_close_frac'] = np.mean(close_mask)
                feats[f'{phase}_tof_{s}_missing_frac'] = np.mean(vals == -1)

        features.append(feats)

    return pd.DataFrame(features)

In [ ]:
features = extract_features(train)

labels = train.groupby('sequence_id').first()[['gesture', 'sequence_type', 'subject', 'orientation']].reset_index()
train_df = features.merge(labels, on='sequence_id')
train_df = train_df.merge(train_demo, on='subject')

X = train_df.drop(columns=['sequence_id', 'gesture','sequence_type', 'subject', 'orientation'])
y = train_df['sequence_type'].map({'Target': 1, 'Non-Target': 0})

X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

binary_model = LGBMClassifier(n_estimators=100, random_state=42)
binary_model.fit(X_train, y_train)

y_pred = binary_model.predict(X_val)
print(classification_report(y_val, y_pred))

In [41]:
X = train_df.drop(columns=['sequence_id', 'gesture', 'sequence_type', 'subject', 'orientation' ])
y = train_df['gesture']

gesture_encoder = LabelEncoder()
orientation_encoder = LabelEncoder()
y_encoded = gesture_encoder.fit_transform(y)
# X['oe'] = orientation_encoder.fit_transform(train_df['orientation'])


X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, stratify=y, test_size=0.2, random_state=42)


In [42]:
model = LGBMClassifier(
    objective='multiclass',
    num_class=len(le.classes_),  # important!
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)
y_pred_encoded = model.predict(X_val)
y_pred = le.inverse_transform(y_pred_encoded)
y_val_decoded = le.inverse_transform(y_val)
print(classification_report(y_val_decoded, y_pred))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001897 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16427
[LightGBM] [Info] Number of data points in the train set: 6520, number of used features: 71
[LightGBM] [Info] Start training from score -2.548219
[LightGBM] [Info] Start training from score -2.550182
[LightGBM] [Info] Start training from score -3.922817
[LightGBM] [Info] Start training from score -2.548219
[LightGBM] [Info] Start training from score -2.544305
[LightGBM] [Info] Start training from score -3.922817
[LightGBM] [Info] Start training from score -2.544305
[LightGBM] [Info] Start training from score -2.544305
[LightGBM] [Info] Start training from score -3.922817
[LightGBM] [Info] Start training from score -2.544305
[LightGBM] [Info] Start training from score -2.544305
[LightGBM] [Info] Start training from score -3.922817
[LightGBM] [Info] Start training from score -2.839830
[LightGBM

,sequence_id,acc_x_Gesture_mean,acc_x_Gesture_std,acc_x_Gesture_max,acc_x_Gesture_min,acc_y_Gesture_mean,acc_y_Gesture_std,acc_y_Gesture_max,acc_y_Gesture_min,acc_z_Gesture_mean,...,Transition_tof_1_close_frac,Transition_tof_1_missing_frac,Transition_tof_2_close_frac,Transition_tof_2_missing_frac,Transition_tof_3_close_frac,Transition_tof_3_missing_frac,Transition_tof_4_close_frac,Transition_tof_4_missing_frac,Transition_tof_5_close_frac,Transition_tof_5_missing_frac
0,SEQ_000007,6.875854,0.779424,9.015625,5.492188,5.640137,0.582177,6.519531,4.683594,4.259521,...,0.030000,0.514375,0.000000,0.603750,0.092500,0.519375,0.077500,0.304375,0.000000,0.456875
1,SEQ_000008,3.449929,0.736728,4.949219,1.734375,7.261364,0.824113,8.667969,4.718750,5.647609,...,0.000000,0.621429,0.000000,0.690179,0.000000,0.592411,0.000000,0.465179,0.000000,0.813393
2,SEQ_000013,-7.591276,1.143683,-5.226562,-9.250000,3.751562,0.571292,4.683594,2.656250,-5.493620,...,0.023098,0.591712,0.063859,0.657609,0.120924,0.348505,0.000000,0.485734,0.008832,0.673913
3,SEQ_000016,5.291590,0.940102,9.378906,3.859375,-4.744830,0.441674,-3.113281,-5.718750,-6.674517,...,0.349537,0.644097,0.385995,0.581019,0.187500,0.776042,0.472222,0.396412,0.243634,0.730903
4,SEQ_000018,6.235639,0.241211,6.832031,5.718750,6.068704,0.211266,6.449219,5.566406,4.739890,...,0.000000,0.521875,0.005469,0.559375,0.000000,0.702344,0.000000,0.378125,0.000000,0.307031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8146,SEQ_065508,-7.286932,0.393596,-6.410156,-8.058594,6.254498,0.255871,6.808594,5.542969,-2.798177,...,0.480741,0.373910,0.427326,0.405887,0.410974,0.432049,0.486919,0.314680,0.561773,0.353198
8147,SEQ_065519,0.811458,0.862646,1.937500,-0.628906,9.518099,0.245501,10.035156,8.769531,-3.321224,...,0.011863,0.702836,0.002025,0.680266,0.227720,0.638310,0.036748,0.702546,0.000000,0.827546
8148,SEQ_065522,-9.020573,0.501440,-7.847656,-10.023438,-3.511719,0.813275,-2.332031,-4.785156,-1.737630,...,0.000326,0.674154,0.002604,0.806641,0.000000,0.838216,0.001628,0.644206,0.000000,0.529297
8149,SEQ_065526,7.894531,0.252330,8.390625,7.324219,6.123958,0.201382,6.671875,5.636719,0.311068,...,0.000000,0.447049,0.000000,0.477431,0.000000,0.506076,0.000000,0.316840,0.000000,0.419271


In [63]:
def test_feature_set(func) -> tuple[LGBMClassifier]:
    # warnings.filterwarnings("ignore", category=UserWarning)
    features = func(train)

    labels = train.groupby('sequence_id').first()[['gesture', 'sequence_type', 'subject', 'orientation']].reset_index()
    train_df = features.merge(labels, on='sequence_id')
    train_df = train_df.merge(train_demo, on='subject')

    X = train_df.drop(columns=['sequence_id', 'gesture','sequence_type', 'subject', 'orientation'])
    y = train_df['sequence_type'].map({'Target': 1, 'Non-Target': 0})

    X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    binary_model = LGBMClassifier(n_estimators=100, random_state=42, verbose=0)
    binary_model.fit(X_train, y_train)

    y_pred = binary_model.predict(X_val)
    print('Binary Model Results')
    print(classification_report(y_val, y_pred))

    X = train_df.drop(columns=['sequence_id', 'gesture', 'sequence_type', 'subject', 'orientation' ])
    y = train_df['gesture'] #.map({'target': 1, 'non-target': 0})  # Binary target

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)  # LightGBM expects integer labels for multiclass

    X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, stratify=y, test_size=0.2, random_state=42)
    multiclass_model = LGBMClassifier(
        objective='multiclass',
        num_class=len(le.classes_),  # important!
        n_estimators=100,
        random_state=42,
        verbose=-1
    )

    multiclass_model.fit(X_train, y_train)
    y_pred_encoded = multiclass_model.predict(X_val)
    y_pred = le.inverse_transform(y_pred_encoded)
    y_val_decoded = le.inverse_transform(y_val)
    print('multiclass model')
    print(classification_report(y_val_decoded, y_pred))
    return binary_model, multiclass_model

In [64]:
binary_model, multiclass_model = test_feature_set(extract_features)

Binary Model Results
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       608
           1       0.98      0.99      0.99      1023

    accuracy                           0.98      1631
   macro avg       0.98      0.98      0.98      1631
weighted avg       0.98      0.98      0.98      1631

multiclass model
                                            precision    recall  f1-score   support

                     Above ear - pull hair       0.81      0.74      0.77       128
                        Cheek - pinch skin       0.60      0.67      0.63       128
                     Drink from bottle/cup       0.97      0.88      0.92        32
                       Eyebrow - pull hair       0.42      0.48      0.45       128
                       Eyelash - pull hair       0.55      0.50      0.52       128
Feel around in tray and pull out an object       1.00      0.84      0.92        32
                  Forehead - pull hairline    

In [66]:
def old_extract_features(df):
    features = []

    for seq_id, group in df.groupby('sequence_id'):
        feats = {'sequence_id': seq_id}

        # Accelerometer features
        for axis in ['acc_x', 'acc_y', 'acc_z']:
            feats[f'{axis}_mean'] = group[axis].mean()
            feats[f'{axis}_std'] = group[axis].std()
            feats[f'{axis}_max'] = group[axis].max()
            feats[f'{axis}_min'] = group[axis].min()

        # Thermopile sensors
        for i in range(1, 6):
            col = f'thm_{i}'
            feats[f'{col}_mean'] = group[col].mean()
            feats[f'{col}_std'] = group[col].std()

        # ToF: Count close pixels & missing data
        for s in range(1, 6):
            tof_cols = [f'tof_{s}_v{v}' for v in range(64)]
            vals = group[tof_cols].values
            close_mask = (vals >= 0) & (vals < 50)
            feats[f'tof_{s}_close_frac'] = np.mean(close_mask)
            feats[f'tof_{s}_missing_frac'] = np.mean(vals == -1)

        features.append(feats)

    return pd.DataFrame(features)




In [67]:
old_binary_model, old_multiclass_model = test_feature_set(old_extract_features)

Binary Model Results
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       608
           1       0.97      0.97      0.97      1023

    accuracy                           0.96      1631
   macro avg       0.96      0.96      0.96      1631
weighted avg       0.96      0.96      0.96      1631

multiclass model
                                            precision    recall  f1-score   support

                     Above ear - pull hair       0.73      0.64      0.68       128
                        Cheek - pinch skin       0.46      0.51      0.49       128
                     Drink from bottle/cup       0.89      0.78      0.83        32
                       Eyebrow - pull hair       0.25      0.27      0.26       128
                       Eyelash - pull hair       0.36      0.38      0.37       128
Feel around in tray and pull out an object       1.00      0.75      0.86        32
                  Forehead - pull hairline    

In [68]:
test = pd.read_csv('data/test.csv')

In [70]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [83]:
train.groupby(['gesture', 'sequence_counter'])['

gesture                sequence_counter
Above ear - pull hair  0                   638
                       1                   638
                       2                   638
                       3                   638
                       4                   638
                                          ... 
Write name on leg      118                   2
                       119                   2
                       120                   2
                       121                   2
                       122                   2
Name: row_id, Length: 4891, dtype: int64